<a href="https://colab.research.google.com/github/Phimphika113/sobeam_year4/blob/main/Pyomo_%E0%B8%81%E0%B9%88%E0%B8%AD%E0%B8%99%E0%B8%9E%E0%B8%9A%E0%B8%AD%E0%B8%B2%E0%B8%88%E0%B8%B2%E0%B8%A3%E0%B8%A2%E0%B9%8C30_AUG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. ติดตั้ง Pyomo และ Solver

In [45]:
!pip install -q pyomo ## ติดตั้ง Pyomo
!apt-get install -y glpk-utils  ##Solver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


##2. นำเข้าไลบรารีที่จำเป็น


In [46]:
from pyomo.environ import *


##3. กำหนดข้อมูลพื้นฐาน

In [47]:
# กำหนดจำนวนพยาบาลและจำนวนวัน
n = 17  # จำนวนพยาบาล
D = 31  # จำนวนวันในเดือน

# สร้างกลุ่มข้อมูล
nurses = range(1, n+1)  # พยาบาล (index: 1 ถึง n)
shifts = [1, 2, 3]      # ผลัด: 1=เช้า, 2=บ่าย, 3=ดึก
days = range(1, D+1)    # จำนวนวันในเดือน (index: 1 ถึง D)


##4. กำหนดข้อมูลของพยาบาลแต่ละคน


In [48]:
# สร้าง dictionary ที่เก็บข้อมูลพยาบาลแต่ละคน
Mij = {(i, j): 0 for i in nurses for j in days}  # พยาบาลขอหยุดทำงานในวันนั้นๆ
Bij = {(i, j): 0 for i in nurses for j in days}  # พยาบาลลาพักร้อน
di = {i: 0 for i in nurses}                          # พยาบาลที่มีโรคประจำตัว
ages = {i: 0 for i in nurses}                      # อายุของพยาบาล
work_experience = {i: 0 for i in nurses}     # อายุการทำงานของพยาบาล


In [49]:
# การกำหนดข้อมูล (สมมติ) กำหนดวันขอหยุด
nurses_days_off = {
    1: [5, 10],          # พยาบาลที่ 1 หยุดวันที่ 5 และ 10
    2: [15],          # พยาบาลที่ 2 หยุดวันที่ 15
    3: [20, 25],
    4: [3, 12],
    5: [8, 18],
    6: [1, 14],
    7: [7],
    8: [16, 24],
    9: [2, 11],
    10: [9, 19],
    11: [4, 22],
    12: [6, 13],
    13: [17],
    14: [21, 26],
    15: [23],
    16: [5, 20],
    17: [12, 27],
    }
# ใช้ลูปเพื่อกำหนดวันหยุดให้กับพยาบาลหลายคน
for nurse_id, days_off in nurses_days_off.items():
    for day in days_off:
        Mij[(nurse_id, day)] = 1

In [50]:
# กำหนดวันลาพักร้อน
nurses_vacation = {


    1: [6, 21],               # พยาบาลที่ 1 ลาพักร้อนวันที่ 6 และ 21
    2: [10, 27],
    3: [4, 15],
    4: [8, 18],
    5: [12],
    6: [14, 22],
    7: [7, 25],
    8: [13],
    9: [9, 24],
    10: [3],
    11: [11, 19],
    12: [16],
    13: [1, 20],
    14: [5, 30],
    15: [17],
    16: [23],
    17: [6, 26],
    }

# ใช้ลูปเพื่อกำหนดวันลาพักร้อน
for nurse_id, vacation_days in nurses_vacation.items():
    for day in vacation_days:
        Bij[(nurse_id, day)] = 1

In [51]:
# กำหนดพยาบาลที่มีโรคประจำตัว
nurses_with_conditions = {

    1: 1,          #พยาบาลที่ 1 มีโรคประจำตัว
    2: 1,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
    7: 0,
    8: 0,
    9: 0,
    10: 0,
    11: 0,
    12: 0,
    13: 0,
    14: 0,
    15: 0,
    16: 0,
    17: 0,
}
# ใช้ลูปเพื่อกำหนดพยาบาลที่มีโรคประจำตัว
for nurse_id, condition in nurses_with_conditions.items():
    di[nurse_id] = condition

In [52]:
# กำหนดอายุพยาบาล
ages = {


    1: 55,  #พยาบาลที่ 1 อายุ 55 ปี
    2: 56,
    3: 45,
    4: 40,
    5: 39,
    6: 38,
    7: 45,
    8: 32,
    9: 36,
    10: 45,
    11: 35,
    12: 37,
    13: 35,
    14: 35,
    15: 24,
    16: 24,
    17: 23,
}
# ใช้ลูปเพื่อกำหนดอายุของพยาบาล
for nurse_id, age in ages.items():
    ages[nurse_id] = age

In [53]:
#กำหนดอายุงานพยาบาล
work_experience = {
    1: 18,  # พยาบาลคนที่ 1 มีอายุการทำงาน 5 ปี
    2: 18,  # พยาบาลคนที่ 2 มีอายุการทำงาน 1 ปี
    3: 15, # พยาบาลคนที่ 3 มีอายุการทำงาน 10 ปี
    4: 14,
    5: 14,
    6: 12,
    7: 15,
    8: 10,
    9: 9,
    10: 12,
    11: 10,
    12: 9,
    13: 8,
    14: 6,
    15: 1,
    16: 1,
    17: 1,
}


##5. สร้างโมเดล

In [54]:
# prompt: # สร้างโมเดล
# model = ConcreteModel()
# # กำหนดตัวแปรตัดสินใจ (Decision Variables)
# model.X = Var(nurses, days, shifts, within=Binary)  # ตัวแปรการทำงานในผลัดปกติ
# model.Z = Var(nurses, days, shifts, within=Binary)  # ตัวแปรการทำงานในผลัดล่วงเวลา
# # กำหนดพารามิเตอร์ใน Pyomo โดยใช้ข้อมูลที่เตรียมไว้
# model.M = Param(nurses, days, initialize=Mij, within=Binary)
# model.B = Param(nurses, days, initialize=Bi

# สร้างโมเดล
model = ConcreteModel()

# กำหนดตัวแปรตัดสินใจ (Decision Variables)
model.X = Var(nurses, days, shifts, within=Binary)  # ตัวแปรการทำงานในผลัดปกติ
model.Z = Var(nurses, days, shifts, within=Binary)  # ตัวแปรการทำงานในผลัดล่วงเวลา

# กำหนดพารามิเตอร์ใน Pyomo โดยใช้ข้อมูลที่เตรียมไว้
model.M = Param(nurses, days, initialize=Mij, within=Binary)
model.B = Param(nurses, days, initialize=Bij, within=Binary)
model.d = Param(nurses, initialize=di, within=Binary)
model.ages = Param(nurses, initialize=ages, within=NonNegativeIntegers)
model.work_experience = Param(nurses, initialize=work_experience, within=NonNegativeIntegers)

# กำหนดตัวแปรที่สำคัญสำหรับการคำนวณ
model.NT = Var(nurses, within=NonNegativeIntegers)  # จำนวนผลัดที่พยาบาลทำงาน
model.OT = Var(nurses, within=NonNegativeIntegers)  # จำนวนผลัดล่วงเวลาทั้งหมดของพยาบาล

# เพิ่มตัวแปรที่ใช้แทนค่า absolute value
model.NT_diff = Var(nurses, within=NonNegativeReals)
model.OT_diff = Var(nurses, within=NonNegativeReals)

In [55]:
# กำหนดตัวแปรที่สำคัญสำหรับการคำนวณ
#model.NT = Var(nurses, within=NonNegativeIntegers)  # จำนวนผลัดที่พยาบาลทำงาน
#model.OT = Var(nurses, within=NonNegativeIntegers)  # จำนวนผลัดล่วงเวลาทั้งหมดของพยาบาล

# NT_i: จำนวนผลัดที่พยาบาล i ทำงานใน 1 เดือน
#def NT_constraint_rule(model, i):
#    return model.NT[i] == sum(model.X[i, j, k] for j in days for k in shifts)
#model.NT_constraint = Constraint(nurses, rule=NT_constraint_rule)

# OT_i: จำนวนผลัดที่พยาบาล i ทำงานล่วงเวลาใน 1 เดือน
#def OT_constraint_rule(model, i):
 #   return model.OT[i] == sum(model.Z[i, j, k] for j in days for k in shifts)
#model.OT_constraint = Constraint(nurses, rule=OT_constraint_rule)


In [56]:

# คำนวณค่าเฉลี่ย
alpha = sum(model.NT[i] for i in nurses) / n  # ค่าเฉลี่ยของ NT
beta = sum(model.OT[i] for i in nurses) / n  # ค่าเฉลี่ยของ OT

In [57]:
# ข้อจำกัดเพื่อให้ตัวแปร NT_diff และ OT_diff แทนค่า abs ได้อย่างถูกต้อง
def nt_diff_constraint_rule(model, i):
    return model.NT_diff[i] >= model.NT[i] - alpha

def ot_diff_constraint_rule(model, i):
    return model.OT_diff[i] >= model.OT[i] - beta

def nt_diff_constraint_rule_neg(model, i):
    return model.NT_diff[i] >= alpha - model.NT[i]

def ot_diff_constraint_rule_neg(model, i):
    return model.OT_diff[i] >= beta - model.OT[i]

model.nt_diff_constraint = Constraint(nurses, rule=nt_diff_constraint_rule)
model.ot_diff_constraint = Constraint(nurses, rule=ot_diff_constraint_rule)
model.nt_diff_constraint_neg = Constraint(nurses, rule=nt_diff_constraint_rule_neg)
model.ot_diff_constraint_neg = Constraint(nurses, rule=ot_diff_constraint_rule_neg)

##6. สมการวัตถุประสงค์


In [58]:
def objective_rule(model):
    return sum(model.NT_diff[i] for i in nurses) + \
           sum(model.OT_diff[i] for i in nurses)

model.objective = Objective(rule=objective_rule, sense=minimize)

##7.ข้อจำกัด (Constraints)

In [59]:
# ข้อจำกัด 3.1: ในแต่ละผลัดต้องมีพยาบาล 5 คน
def constraint_3_1_rule(model, j, k):
    return sum(model.X[i, j, k] for i in nurses) == 5
model.constraint_3_1 = Constraint(days, shifts, rule=constraint_3_1_rule)


In [44]:
# ข้อจำกัด 3.2: พยาบาลแต่ละคนทำงานไม่เกิน 5 ผลัดต่อสัปดาห์
def constraint_3_2_rule(model, i, g):
    return sum(model.X[i, j, k] for j in range(g, g+5) for k in shifts) <= 5
model.constraint_3_2 = Constraint(nurses, range(1, D-4), rule=constraint_3_2_rule)


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
# ข้อจำกัด 3.3: ไม่สามารถหยุดพร้อมกันเกิน 4 คนในแต่ละผลัด
def constraint_3_3_rule(model, j, k):
    return sum(1 - model.X[i, j, k] for i in nurses) <= 4
model.constraint_3_3 = Constraint(days, shifts, rule=constraint_3_3_rule)

In [ ]:
# ข้อจำกัด 3.4: ห้ามหยุดงานติดกันเกิน 4 วัน
def constraint_3_4_rule(model, i, g):
    return sum(1 - model.X[i, j, k] for j in range(g, g+5) for k in shifts) <= 4
model.constraint_3_4 = Constraint(nurses, range(1, D-4), rule=constraint_3_4_rule)

In [ ]:
# ข้อจำกัด 3.5: พยาบาลอายุ 50 ปีขึ้นไปต้องทำงานผลัดเช้ามากกว่าผลัดบ่าย
age_over_50_or_with_condition = [i for i in nurses if ages[i] >= 50]  # พยาบาลที่อายุ 50 ปีขึ้นไป
def constraint_3_5_rule(model, i):
    if i in age_over_50_or_with_condition:
        return sum(model.X[i, j, 1] for j in days) >= sum(model.X[i, j, 2] for j in days)+1
    else:
        return Constraint.Skip
model.constraint_3_5 = Constraint(nurses, rule=constraint_3_5_rule)

In [ ]:
# ข้อจำกัด 3.6: พยาบาลที่มีอายุ 50 ปีขึ้นไป และพยาบาลที่มีโรคประจำตัวต้องห้ามขึ้นเวรดึก (k = 3)
def constraint_3_6_rule(model, i, j):
    if ages[i] >= 50 or di[i] == 1:
        return model.X[i, j, 3] == 0  # ห้ามทำงานเวรดึก
    else:
        return Constraint.Skip
model.constraint_3_6 = Constraint(nurses, days, rule=constraint_3_6_rule)

In [ ]:
# ข้อจำกัด 3.7: พยาบาลห้ามขึ้นเวรดึกติดต่อกันในวันถัดไป
def constraint_3_7_rule(model, i, j):
    if j < D:  # วันสุดท้ายไม่ต้องเช็ค
        return model.X[i, j, 3] + model.X[i, j+1, 3] <= 1
    else:
        return Constraint.Skip
model.constraint_3_7 = Constraint(nurses, days, rule=constraint_3_7_rule)

In [ ]:
# ข้อจำกัด 3.8: ห้ามพยาบาลที่มีอายุการทำงานไม่เกิน 1 ปี ขึ้นผลัดการทำงานร่วมกัน
def constraint_3_8_rule(model, j, k):
    return sum(model.X[i, j, k] for i in nurses if work_experience[i] <= 1) <= 1
model.constraint_3_8 = Constraint(days, shifts, rule=constraint_3_8_rule)


#8.เรียกใช้ Solver เพื่อแก้ปัญหา

In [60]:
solver = SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 0.0, 'Upper bound': 0.0, 'Number of objectives': 1, 'Number of constraints': 161, 'Number of variables': 1649, 'Number of nonzeros': 2805, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.018416881561279297}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

#9.ฟังก์ชันแสดงผลลัพธ์

In [61]:
import pandas as pd
from pyomo.environ import ConcreteModel, Var, Param, Constraint, Binary, NonNegativeIntegers, Objective, SolverFactory, NonNegativeReals

# ... (โค้ดเดิมของคุณสำหรับการสร้างโมเดล)

# แก้ไขโมเดลโดยใช้ solver
solver = SolverFactory('glpk')  # หรือ solver อื่นๆ ที่คุณต้องการใช้
results = solver.solve(model)

# สร้าง DataFrame ว่างสำหรับเก็บตารางการทำงาน
schedule = pd.DataFrame(index=nurses, columns=days)

# กำหนดชื่อของผลัด
shifts = {1: "เช้า", 2: "บ่าย", 3: "ดึก"}

# เติมข้อมูลลงในตาราง
for i in nurses:
    for j in days:
        for k in shifts:
            if model.X[i, j, k].value == 1:  # ถ้าพยาบาลทำงานในผลัดนั้น
                schedule.at[i, j] = shifts[k]

# แสดงตารางเวลา
print(schedule)

     1    2    3    4    5    6    7    8    9    10  ...   22   23   24   25  \
1   ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ...  ดึก  ดึก  ดึก  ดึก   
2   ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ...  ดึก  ดึก  ดึก  ดึก   
3   ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ...  ดึก  ดึก  ดึก  ดึก   
4   ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ดึก  ...  ดึก  ดึก  ดึก  ดึก   
5   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN   
6   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN   
7   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN   
8   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN   
9   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN   
10  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN   
11  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN   
12  NaN  NaN  NaN  NaN  NaN 

In [ ]:
result = solver.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp1vke678_.glpk.raw --wglp /tmp/tmpzafiv4ef.glpk.glp --cpxlp
 /tmp/tmpde9jer8u.pyomo.lp
Reading problem data from '/tmp/tmpde9jer8u.pyomo.lp'...
/tmp/tmpde9jer8u.pyomo.lp:26274: warning: lower bound of variable 'x139' redefined
/tmp/tmpde9jer8u.pyomo.lp:26274: warning: upper bound of variable 'x139' redefined
1805 rows, 1649 columns, 19131 non-zeros
1615 integer variables, 1581 of which are binary
27855 lines were read
Writing problem data to '/tmp/tmpzafiv4ef.glpk.glp'...
24433 lines were written
GLPK Integer Optimizer 5.0
1805 rows, 1649 columns, 19131 non-zeros
1615 integer variables, 1581 of which are binary
Preprocessing...
PROBLEM HAS NO PRIMAL FEASIBLE SOLUTION
Time used:   0.0 secs
Memory used: 2.8 Mb (2988004 bytes)
Writing MIP solution to '/tmp/tmp1vke678_.glpk.raw'...
3463 lines were written
